# 05. 音声処理

人の音声の分析と処理技術を学びます。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. 基本周波数（ピッチ）追跡

In [ ]:
class VoiceProcessor:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        
    def create_voice_like_signal(self, duration=3.0):
        """音声に似た信号を生成"""
        t = np.linspace(0, duration, int(self.sample_rate * duration), False)
        
        # 基本周波数の変化（男性の話し声：80-200Hz）
        f0_base = 120  # Hz
        f0_variation = 30 * np.sin(2 * np.pi * 0.5 * t)  # 抑揚
        f0 = f0_base + f0_variation
        
        # 累積位相
        phase = np.cumsum(2 * np.pi * f0 / self.sample_rate)
        
        # 声帯振動（のこぎり波ベース）
        glottal_pulse = signal.sawtooth(phase)
        
        # フォルマント（口の形による共鳴）
        # フォルマント1: 700Hz, フォルマント2: 1220Hz, フォルマント3: 2600Hz
        formants = [700, 1220, 2600]
        formant_gains = [1.0, 0.7, 0.3]
        
        voice_signal = np.zeros_like(t)
        
        for formant_freq, gain in zip(formants, formant_gains):
            # バンドパスフィルタでフォルマントを作成
            nyquist = self.sample_rate / 2
            low = (formant_freq - 100) / nyquist
            high = (formant_freq + 100) / nyquist
            
            if high < 1.0:
                b, a = signal.butter(2, [low, high], btype='band')
                formant_signal = signal.filtfilt(b, a, glottal_pulse)
                voice_signal += gain * formant_signal
        
        # 振幅エンベロープ（発話の区切り）
        envelope = np.ones_like(t)
        
        # 無音区間を追加
        silence_1 = (t > 0.8) & (t < 1.0)
        silence_2 = (t > 2.0) & (t < 2.3)
        envelope[silence_1] = 0.1
        envelope[silence_2] = 0.1
        
        voice_signal *= envelope
        
        return t, voice_signal, f0
        
    def pitch_tracking_autocorr(self, audio, frame_size=1024, hop_size=256):
        """自己相関によるピッチ追跡"""
        pitches = []
        confidences = []
        times = []
        
        for i in range(0, len(audio) - frame_size, hop_size):
            frame = audio[i:i + frame_size]
            time = i / self.sample_rate
            times.append(time)
            
            # 自己相関
            autocorr = np.correlate(frame, frame, mode='full')
            autocorr = autocorr[len(autocorr)//2:]
            
            # 正規化
            if autocorr[0] > 0:
                autocorr = autocorr / autocorr[0]
            
            # ピッチ範囲（80-400Hz）
            min_period = int(self.sample_rate / 400)
            max_period = int(self.sample_rate / 80)
            
            if max_period < len(autocorr):
                search_range = autocorr[min_period:max_period]
                
                if len(search_range) > 0:
                    max_idx = np.argmax(search_range)
                    period = max_idx + min_period
                    pitch = self.sample_rate / period
                    confidence = search_range[max_idx]
                else:
                    pitch = 0
                    confidence = 0
            else:
                pitch = 0
                confidence = 0
                
            pitches.append(pitch)
            confidences.append(confidence)
        
        return np.array(times), np.array(pitches), np.array(confidences)
        
    def formant_analysis(self, audio, frame_size=1024):
        """フォルマント分析（簡易版）"""
        # LPC（線形予測符号化）の簡易実装
        windowed = audio * np.hanning(len(audio))
        
        # FFTによるスペクトル
        spectrum = np.abs(fft(windowed))
        freqs = fftfreq(len(windowed), 1/self.sample_rate)
        
        # 正の周波数のみ
        positive_mask = freqs >= 0
        freqs_pos = freqs[positive_mask]
        spectrum_pos = spectrum[positive_mask]
        
        # ピーク検出でフォルマント推定
        peaks, _ = signal.find_peaks(spectrum_pos, 
                                   height=np.max(spectrum_pos) * 0.1,
                                   distance=int(200 / (self.sample_rate / len(spectrum_pos))))
        
        formant_freqs = freqs_pos[peaks]
        formant_amps = spectrum_pos[peaks]
        
        # 低い順にソート
        sorted_indices = np.argsort(formant_freqs)
        formant_freqs = formant_freqs[sorted_indices]
        formant_amps = formant_amps[sorted_indices]
        
        return formant_freqs[:3], formant_amps[:3]  # 最初の3つのフォルマント

# 音声処理のデモ
voice_proc = VoiceProcessor()

# 音声様信号の生成
t_voice, voice_signal, true_f0 = voice_proc.create_voice_like_signal()

# ピッチ追跡
pitch_times, detected_pitches, pitch_confidences = voice_proc.pitch_tracking_autocorr(voice_signal)

# フォルマント分析（音声の中間部分）
mid_start = len(voice_signal) // 3
mid_end = 2 * len(voice_signal) // 3
formant_freqs, formant_amps = voice_proc.formant_analysis(voice_signal[mid_start:mid_end])

plt.figure(figsize=(16, 12))

# 音声波形
plt.subplot(3, 2, 1)
plt.plot(t_voice, voice_signal, 'b-', linewidth=1)
plt.title('合成音声信号')
plt.ylabel('振幅')
plt.grid(True)

# スペクトログラム
plt.subplot(3, 2, 2)
f, t_spec, Sxx = signal.spectrogram(voice_signal, voice_proc.sample_rate, 
                                   nperseg=512, noverlap=256)
plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx + 1e-10), 
              shading='gouraud', cmap='viridis')
plt.title('音声スペクトログラム')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 3000)
plt.colorbar(label='振幅 (dB)')

# フォルマントをマーク
for formant_freq in formant_freqs:
    plt.axhline(y=formant_freq, color='red', linestyle='--', alpha=0.8, linewidth=2)

# ピッチ追跡結果
plt.subplot(3, 2, 3)
# 信頼度によるフィルタリング
confident_mask = pitch_confidences > 0.3
plt.plot(pitch_times, detected_pitches, 'ro-', markersize=4, alpha=0.7, label='検出ピッチ')
plt.plot(pitch_times[confident_mask], detected_pitches[confident_mask], 
         'go-', markersize=6, label='高信頼度')

# 真のピッチをプロット
t_interp = np.interp(pitch_times, t_voice, true_f0)
plt.plot(pitch_times, t_interp, 'b-', linewidth=2, label='真のピッチ')

plt.title('ピッチ追跡結果')
plt.xlabel('時間 (秒)')
plt.ylabel('基本周波数 (Hz)')
plt.legend()
plt.grid(True)
plt.ylim(50, 250)

# ピッチ信頼度
plt.subplot(3, 2, 4)
plt.plot(pitch_times, pitch_confidences, 'g-', linewidth=2)
plt.axhline(y=0.3, color='r', linestyle='--', label='信頼度しきい値')
plt.title('ピッチ検出信頼度')
plt.xlabel('時間 (秒)')
plt.ylabel('信頼度')
plt.legend()
plt.grid(True)

# フォルマント分析結果
plt.subplot(3, 2, 5)
mid_spectrum = np.abs(fft(voice_signal[mid_start:mid_end] * 
                         np.hanning(mid_end - mid_start)))
mid_freqs = fftfreq(mid_end - mid_start, 1/voice_proc.sample_rate)
positive_mask = mid_freqs >= 0

plt.plot(mid_freqs[positive_mask], 20 * np.log10(mid_spectrum[positive_mask] + 1e-10), 
         'b-', linewidth=1)

# 検出されたフォルマントをマーク
for i, (freq, amp) in enumerate(zip(formant_freqs, formant_amps)):
    plt.axvline(x=freq, color='red', linestyle='--', alpha=0.8)
    plt.text(freq, 20 * np.log10(amp + 1e-10) + 5, f'F{i+1}\n{freq:.0f}Hz', 
            ha='center', fontsize=10, color='red')

plt.title('フォルマント分析')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.grid(True)
plt.xlim(0, 3500)

# 統計情報
plt.subplot(3, 2, 6)
plt.axis('off')

# ピッチ統計
valid_pitches = detected_pitches[confident_mask]
if len(valid_pitches) > 0:
    pitch_mean = np.mean(valid_pitches)
    pitch_std = np.std(valid_pitches)
    pitch_range = np.max(valid_pitches) - np.min(valid_pitches)
else:
    pitch_mean = pitch_std = pitch_range = 0

stats_text = f"""
音声分析結果

ピッチ統計:
  平均: {pitch_mean:.1f} Hz
  標準偏差: {pitch_std:.1f} Hz
  範囲: {pitch_range:.1f} Hz
  
フォルマント:
"""

for i, freq in enumerate(formant_freqs):
    stats_text += f"  F{i+1}: {freq:.0f} Hz\n"

stats_text += f"""
品質指標:
  ピッチ検出率: {np.mean(confident_mask)*100:.1f}%
  平均信頼度: {np.mean(pitch_confidences):.2f}
"""

plt.text(0.05, 0.95, stats_text, fontsize=12, verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue", alpha=0.8))

plt.tight_layout()
plt.show()

print("音声分析の要素:")
print("1. 基本周波数（F0）: 声の高さ")
print("2. フォルマント: 母音の特徴")
print("3. ピッチ追跡: 抑揚の分析")
print("4. 信頼度評価: 検出精度の指標")

## 2. 音声強調と雑音除去

In [ ]:
class VoiceEnhancement:
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        
    def spectral_subtraction(self, noisy_signal, noise_segment, alpha=2.0):
        """スペクトル減算による雑音除去"""
        # ノイズのスペクトル推定
        noise_spectrum = np.abs(fft(noise_segment * np.hanning(len(noise_segment))))
        noise_power = noise_spectrum ** 2
        
        # フレームごとの処理
        frame_size = len(noise_segment)
        hop_size = frame_size // 2
        enhanced_signal = np.zeros_like(noisy_signal)
        
        for i in range(0, len(noisy_signal) - frame_size, hop_size):
            frame = noisy_signal[i:i + frame_size]
            windowed_frame = frame * np.hanning(frame_size)
            
            # FFT
            frame_fft = fft(windowed_frame)
            frame_magnitude = np.abs(frame_fft)
            frame_phase = np.angle(frame_fft)
            frame_power = frame_magnitude ** 2
            
            # スペクトル減算
            enhanced_power = frame_power - alpha * noise_power
            
            # 負の値を制限（オーバーサブトラクション防止）
            enhanced_power = np.maximum(enhanced_power, 0.1 * frame_power)
            
            # 振幅の復元
            enhanced_magnitude = np.sqrt(enhanced_power)
            
            # 逆FFT
            enhanced_fft = enhanced_magnitude * np.exp(1j * frame_phase)
            enhanced_frame = np.real(np.fft.ifft(enhanced_fft))
            
            # オーバーラップアッド
            end_idx = min(i + frame_size, len(enhanced_signal))
            enhanced_signal[i:end_idx] += enhanced_frame[:end_idx-i] * np.hanning(frame_size)[:end_idx-i]
        
        return enhanced_signal
        
    def wiener_filter(self, noisy_signal, noise_segment):
        """ウィーナーフィルタによる雑音除去"""
        # ノイズパワー推定
        noise_spectrum = np.abs(fft(noise_segment * np.hanning(len(noise_segment))))
        noise_power = np.mean(noise_spectrum ** 2)
        
        # 信号全体のFFT
        signal_fft = fft(noisy_signal * np.hanning(len(noisy_signal)))
        signal_power = np.abs(signal_fft) ** 2
        
        # ウィーナーフィルタの係数
        wiener_gain = signal_power / (signal_power + noise_power)
        
        # フィルタリング
        enhanced_fft = signal_fft * wiener_gain
        enhanced_signal = np.real(np.fft.ifft(enhanced_fft))
        
        return enhanced_signal
        
    def preemphasis(self, signal, alpha=0.97):
        """プリエンファシス（高周波強調）"""
        return np.append(signal[0], signal[1:] - alpha * signal[:-1])
        
    def deemphasis(self, signal, alpha=0.97):
        """ディエンファシス（プリエンファシスの逆）"""
        deemphasized = np.zeros_like(signal)
        deemphasized[0] = signal[0]
        for i in range(1, len(signal)):
            deemphasized[i] = signal[i] + alpha * deemphasized[i-1]
        return deemphasized
        
    def voice_activity_detection(self, signal, frame_size=512, threshold=0.02):
        """音声区間検出（VAD）"""
        hop_size = frame_size // 2
        vad_flags = []
        times = []
        
        for i in range(0, len(signal) - frame_size, hop_size):
            frame = signal[i:i + frame_size]
            time = i / self.sample_rate
            times.append(time)
            
            # エネルギーベースのVAD
            energy = np.sum(frame ** 2) / frame_size
            
            # ゼロクロッシング率
            zcr = np.sum(np.diff(np.sign(frame)) != 0) / (frame_size - 1)
            
            # 判定（エネルギーとZCRの組み合わせ）
            is_voice = (energy > threshold) and (zcr < 0.3)
            vad_flags.append(is_voice)
            
        return np.array(times), np.array(vad_flags)

# 音声強調のデモ
enhancer = VoiceEnhancement()

# クリーンな音声信号を生成
clean_signal = voice_signal.copy()

# 様々なノイズを追加
np.random.seed(42)

# 1. ホワイトノイズ
white_noise = np.random.normal(0, 0.1, len(clean_signal))
noisy_white = clean_signal + white_noise

# 2. 60Hzハム
hum_noise = 0.05 * np.sin(2 * np.pi * 60 * t_voice)
noisy_hum = clean_signal + hum_noise

# 3. 複合ノイズ
pink_noise = np.random.normal(0, 0.08, len(clean_signal))
# 簡易ピンクノイズフィルタ
b_pink, a_pink = signal.butter(1, 0.1, btype='low')
pink_noise = signal.filtfilt(b_pink, a_pink, pink_noise)
complex_noise = white_noise * 0.5 + hum_noise + pink_noise
noisy_complex = clean_signal + complex_noise

# ノイズセグメント（最初の0.3秒をノイズのみと仮定）
noise_segment_length = int(0.3 * enhancer.sample_rate)
noise_segment = complex_noise[:noise_segment_length]

# 雑音除去処理
enhanced_spectral = enhancer.spectral_subtraction(noisy_complex, noise_segment, alpha=2.0)
enhanced_wiener = enhancer.wiener_filter(noisy_complex, noise_segment)

# プリエンファシス処理
preemphasized = enhancer.preemphasis(clean_signal)
deemphasized = enhancer.deemphasis(preemphasized)

# VAD
vad_times, vad_flags = enhancer.voice_activity_detection(clean_signal)

plt.figure(figsize=(18, 15))

# 元信号とノイズ付き信号
plt.subplot(4, 3, 1)
plt.plot(t_voice, clean_signal, 'b-', linewidth=1, label='クリーン')
plt.plot(t_voice, noisy_complex, 'r-', linewidth=1, alpha=0.7, label='ノイズ付き')
plt.title('元信号 vs ノイズ付き信号')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# スペクトル比較
plt.subplot(4, 3, 2)
clean_spectrum = np.abs(fft(clean_signal * np.hanning(len(clean_signal))))
noisy_spectrum = np.abs(fft(noisy_complex * np.hanning(len(noisy_complex))))
freqs = fftfreq(len(clean_signal), 1/enhancer.sample_rate)
positive_mask = freqs >= 0

plt.plot(freqs[positive_mask], 20 * np.log10(clean_spectrum[positive_mask] + 1e-10), 
         'b-', label='クリーン', linewidth=2)
plt.plot(freqs[positive_mask], 20 * np.log10(noisy_spectrum[positive_mask] + 1e-10), 
         'r-', alpha=0.7, label='ノイズ付き', linewidth=2)
plt.title('スペクトル比較')
plt.xlabel('周波数 (Hz)')
plt.ylabel('振幅 (dB)')
plt.legend()
plt.grid(True)
plt.xlim(0, 4000)

# スペクトル減算結果
plt.subplot(4, 3, 3)
plt.plot(t_voice, clean_signal, 'b-', linewidth=1, label='クリーン')
plt.plot(t_voice[:len(enhanced_spectral)], enhanced_spectral, 'g-', 
         linewidth=1, label='スペクトル減算')
plt.title('スペクトル減算による雑音除去')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# ウィーナーフィルタ結果
plt.subplot(4, 3, 4)
plt.plot(t_voice, clean_signal, 'b-', linewidth=1, label='クリーン')
plt.plot(t_voice[:len(enhanced_wiener)], enhanced_wiener, 'm-', 
         linewidth=1, label='ウィーナーフィルタ')
plt.title('ウィーナーフィルタによる雑音除去')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# 強調処理のスペクトログラム比較
plt.subplot(4, 3, 5)
f_noisy, t_noisy, Sxx_noisy = signal.spectrogram(noisy_complex, enhancer.sample_rate, 
                                                 nperseg=256, noverlap=128)
plt.pcolormesh(t_noisy, f_noisy, 10 * np.log10(Sxx_noisy + 1e-10), 
              shading='gouraud', cmap='Reds')
plt.title('ノイズ付き（スペクトログラム）')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 3000)
plt.colorbar(label='振幅 (dB)')

plt.subplot(4, 3, 6)
# スペクトル減算後のスペクトログラム
if len(enhanced_spectral) > 256:
    f_enh, t_enh, Sxx_enh = signal.spectrogram(enhanced_spectral, enhancer.sample_rate, 
                                              nperseg=256, noverlap=128)
    plt.pcolormesh(t_enh, f_enh, 10 * np.log10(Sxx_enh + 1e-10), 
                  shading='gouraud', cmap='Greens')
plt.title('スペクトル減算後（スペクトログラム）')
plt.ylabel('周波数 (Hz)')
plt.ylim(0, 3000)
plt.colorbar(label='振幅 (dB)')

# プリエンファシス効果
plt.subplot(4, 3, 7)
plt.plot(t_voice, clean_signal, 'b-', linewidth=1, label='元信号')
plt.plot(t_voice[:len(preemphasized)], preemphasized, 'orange', 
         linewidth=1, label='プリエンファシス後')
plt.title('プリエンファシス処理')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# VAD結果
plt.subplot(4, 3, 8)
plt.plot(t_voice, clean_signal, 'b-', linewidth=1, alpha=0.7, label='音声信号')

# VADフラグを音声信号に重ねて表示
for i, (time, is_voice) in enumerate(zip(vad_times, vad_flags)):
    color = 'green' if is_voice else 'red'
    alpha = 0.3
    
    # 時間区間をハイライト
    if i < len(vad_times) - 1:
        next_time = vad_times[i + 1]
        plt.axvspan(time, next_time, color=color, alpha=alpha)

plt.title('音声区間検出（VAD）\n緑：音声, 赤：無音')
plt.xlabel('時間 (秒)')
plt.ylabel('振幅')
plt.legend()
plt.grid(True)

# 処理結果比較（SNR計算）
plt.subplot(4, 3, 9)

def calculate_snr(clean, noisy):
    signal_power = np.mean(clean ** 2)
    noise_power = np.mean((noisy - clean) ** 2)
    if noise_power > 0:
        return 10 * np.log10(signal_power / noise_power)
    else:
        return float('inf')

# SNR計算
snr_noisy = calculate_snr(clean_signal, noisy_complex)
snr_spectral = calculate_snr(clean_signal, enhanced_spectral[:len(clean_signal)])
snr_wiener = calculate_snr(clean_signal, enhanced_wiener[:len(clean_signal)])

methods = ['ノイズ付き', 'スペクトル減算', 'ウィーナーフィルタ']
snr_values = [snr_noisy, snr_spectral, snr_wiener]
colors = ['red', 'green', 'magenta']

bars = plt.bar(methods, snr_values, color=colors, alpha=0.7)
plt.title('SNR改善効果')
plt.ylabel('SNR (dB)')
plt.xticks(rotation=45)

for bar, snr in zip(bars, snr_values):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
            f'{snr:.1f}', ha='center', va='bottom', fontsize=10)

plt.grid(True, axis='y')

# 統計情報
plt.subplot(4, 3, (10, 12))
plt.axis('off')

stats_text = f"""
音声強調処理結果

SNR改善:
  元のSNR: {snr_noisy:.1f} dB
  スペクトル減算後: {snr_spectral:.1f} dB
  ウィーナーフィルタ後: {snr_wiener:.1f} dB
  
改善量:
  スペクトル減算: {snr_spectral - snr_noisy:.1f} dB
  ウィーナーフィルタ: {snr_wiener - snr_noisy:.1f} dB

VAD結果:
  音声率: {np.mean(vad_flags)*100:.1f}%
  無音率: {(1-np.mean(vad_flags))*100:.1f}%

処理の特徴:
  • スペクトル減算: 定常ノイズに効果的
  • ウィーナーフィルタ: より自然な音質
  • プリエンファシス: 高周波強調
  • VAD: 音声区間の自動検出
"""

plt.text(0.05, 0.95, stats_text, fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow", alpha=0.8))

plt.tight_layout()
plt.show()

print("音声強調技術:")
print("1. スペクトル減算: ノイズスペクトルを推定して除去")
print("2. ウィーナーフィルタ: 最適化された雑音除去")
print("3. プリエンファシス: 高周波成分の強調")
print("4. VAD: 音声区間の自動検出")
print("5. SNR: 信号対雑音比による性能評価")

## 練習問題

1. より高精度なピッチ追跡アルゴリズム（YIN、CREPE等）を実装してみましょう
2. 適応的な雑音除去フィルタを作成してみましょう
3. 音声のフォルマント追跡による母音認識システムを構築してみましょう